In [1]:
import torch
import torch.nn as nn

In [51]:
class VAELoss(nn.Module):
    def __init__(self, loss_fn):
        super(VAELoss, self).__init__()
        self.criterion = loss_fn

    # question: how is the loss function using the mu and variance?
    def forward(self, mu, log_var, render_imgs, render_sils, target_imgs, target_sils):
        """gives the batch normalized Variational Error."""

        batch_size = x.shape[0]
        
        sil_err = self.criterion(render_sils, target_sils).abs().mean()
        color_err = self.criterion(render_imgs, target_imgs).abs().mean()
        
        # 0.5 * sum(1 + log(sigma^2) - mu^2 - sigma^2)
        KLD_element = mu.pow(2).add_(log_var.exp()).mul_(-1).add_(1).add_(log_var)
        KL_divergence = torch.sum(KLD_element).mul_(-0.5)

        loss = color_err + sil_err + KL_divergence
        
        return (loss / batch_size).mean()

In [52]:
def huber_loss(x, y, scaling=0.1):
    diff_sq = (x-y) ** 2
    loss = ((1 + diff_sq / (scaling ** 2)).clamp(1e-4).sqrt() - 1) * float(scaling)
    return loss

In [53]:
render_imgs, render_sils = torch.randn(4, 16, 1, 144, 144, requires_grad=True), torch.randn(4, 16, 1, 144, 144, requires_grad=True)
target_imgs, target_sils = torch.randn(4, 16, 1, 144, 144), torch.randn(4, 16, 1, 144, 144)

mu = torch.randn(4, 2048, 1, 1, 8, requires_grad=True)
logvar = torch.randn(4, 2048, 1, 1, 8, requires_grad=True)

In [54]:
los = VAELoss(loss_fn=huber_loss)

In [56]:
loss = los(mu, logvar, render_imgs, render_sils, target_imgs, target_sils)
loss.mean().item()

1620375.125